In [1]:
import numpy as np
import pandas as pd
from full_player_stats import get_dataframe, calculate_points
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [12]:
ind_variables = [
        'FGM', 'FGA', 'FG%', '3PTM', '3PA', '3P%', 'FTM', 'FTA', 'Points', 
       'FT%', 'ORB', 'DRB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS','+/-', 
       'AvPoints', 'RollBckw1', 'RollBckw2'
]

dep_variables = 'RollFwrd1'
        # ,'RollFwrd3'

def get_dataframe(refresh: bool = False) -> pd.DataFrame:
    if refresh is False:
        data = pd.read_csv("all_stats.csv", index_col="Unnamed: 0")
        data.reset_index(inplace=True, drop=True)
        return data

data = get_dataframe(refresh=False)
data = calculate_points(data, roll_backwards=[1,2], roll_forward=[1,3])
data = data.loc[~data[dep_variables].isnull()]
X, y = data[ind_variables], data[dep_variables]


In [13]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [14]:

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))



RMSE: 15.651857


In [16]:
data["preds"] = xg_reg.predict(X)

In [18]:
data.loc[data["Player"] == "Joe Harris"]

,Player,MP,FGM,FGA,FG%,3PTM,3PA,3P%,FTM,FTA,...,PTS,+/-,GameDay,Points,AvPoints,RollBckw1,RollBckw2,RollFwrd1,RollFwrd3,preds
12,Joe Harris,30:46,3.0,9.0,0.333,3.0,5.0,0.600,0.0,0.0,...,9.0,0.0,2021-10-19,11.0,11.000000,11.0,NaN,15.0,14.333333,8.639088
571,Joe Harris,33:02,5.0,14.0,0.357,3.0,10.0,0.300,1.0,2.0,...,14.0,-3.0,2021-10-22,15.0,13.000000,15.0,13.0,10.0,17.333333,13.712770
950,Joe Harris,27:46,3.0,6.0,0.500,1.0,3.0,0.333,0.0,0.0,...,7.0,-17.0,2021-10-24,10.0,12.000000,10.0,12.5,18.0,15.666667,10.350108
1224,Joe Harris,30:22,3.0,12.0,0.250,3.0,11.0,0.273,2.0,2.0,...,11.0,13.0,2021-10-25,18.0,13.500000,18.0,14.0,24.0,16.000000,15.037364
1497,Joe Harris,33:31,5.0,15.0,0.333,5.0,11.0,0.455,0.0,0.0,...,15.0,-22.0,2021-10-27,24.0,15.600000,24.0,21.0,5.0,21.333333,14.315930
1927,Joe Harris,30:45,3.0,9.0,0.333,1.0,6.0,0.167,2.0,2.0,...,9.0,-7.0,2021-10-29,5.0,13.833333,5.0,14.5,19.0,25.333333,12.243570
2435,Joe Harris,25:54,5.0,9.0,0.556,2.0,4.0,0.500,0.0,0.0,...,12.0,15.0,2021-10-31,19.0,14.571429,19.0,12.0,40.0,26.333333,12.489339
2990,Joe Harris,34:28,6.0,10.0,0.600,6.0,8.0,0.750,0.0,0.0,...,18.0,-10.0,2021-11-03,40.0,17.750000,40.0,29.5,17.0,20.333333,13.938859
3270,Joe Harris,28:23,3.0,6.0,0.500,1.0,4.0,0.250,0.0,0.0,...,7.0,17.0,2021-11-05,17.0,17.666667,17.0,28.5,22.0,20.000000,12.968645
3667,Joe Harris,32:13,4.0,8.0,0.500,3.0,7.0,0.429,0.0,0.0,...,11.0,17.0,2021-11-07,22.0,18.100000,22.0,19.5,22.0,28.000000,12.161297
